In [1]:
import pandas as pd
import numpy as np
import os
import glob

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.cluster import DBSCAN
import sklearn.utils
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
from sklearn.metrics.pairwise import haversine_distances

from math import radians


In [2]:
loon =  'N235LB'

os.chdir("/Users/toni/Documents/TFG/og_data/DBSCAN/epsRange/" + loon)
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

raw_df = pd.concat([pd.read_csv(f) for f in all_filenames ])


In [3]:
bl = raw_df.copy()

bl['max_distance'] = bl['max_distance'].astype(str).str[:4]
bl.max_distance.unique()

# USER INPUT!!!
chosen_distance = '4.28'

#-------------------------------

bl = bl.loc[bl.max_distance == chosen_distance]
bl



,ts,aircraft_id,alt,speed,Direction,registration_id,lat,lon,prev_lat,prev_lon,prev_air,prev_ts,distance_since_last_record,time_since_last_record,cluster,eps,max_distance
0,2019-12-05 01:59:21+00:00,HBAL102,500,9,288,N235LB,18.246094,-65.639732,NaN,NaN,NaN,NaN,0.000000,0 days 00:00:00.000000000,0,0.000673,4.28
1,2019-12-05 01:59:28+00:00,HBAL102,575,9,288,N235LB,18.246197,-65.639961,18.246094,-65.639732,HBAL102,2019-12-05 01:59:21+00:00,0.026758,0 days 00:00:07.000000000,0,0.000673,4.28
2,2019-12-05 01:59:34+00:00,HBAL102,650,9,288,N235LB,18.246277,-65.640266,18.246197,-65.639961,HBAL102,2019-12-05 01:59:28+00:00,0.033415,0 days 00:00:06.000000000,0,0.000673,4.28
3,2019-12-05 01:59:40+00:00,HBAL102,700,10,286,N235LB,18.246336,-65.640511,18.246277,-65.640266,HBAL102,2019-12-05 01:59:34+00:00,0.026692,0 days 00:00:06.000000000,0,0.000673,4.28
4,2019-12-05 01:59:52+00:00,HBAL102,825,9,288,N235LB,18.246523,-65.641060,18.246336,-65.640511,HBAL102,2019-12-05 01:59:40+00:00,0.061593,0 days 00:00:12.000000000,0,0.000673,4.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150317,2020-06-23 10:09:07+00:00,HBAL194,51500,23,177,N235LB,18.321396,-107.553749,18.325378,-107.554047,HBAL194,2020-06-23 10:08:30+00:00,0.443894,0 days 00:00:37.000000000,10,0.000673,4.28
150318,2020-06-23 10:15:21+00:00,HBAL194,51600,24,170,N235LB,18.280739,-107.549263,18.321396,-107.553749,HBAL194,2020-06-23 10:09:07+00:00,4.545590,0 days 00:06:14.000000000,10,0.000673,4.28
150319,2020-06-23 10:16:23+00:00,HBAL194,51500,24,168,N235LB,18.273743,-107.547913,18.280739,-107.549263,HBAL194,2020-06-23 10:15:21+00:00,0.790871,0 days 00:01:02.000000000,10,0.000673,4.28
150320,2020-06-23 12:24:33+00:00,HBAL194,51800,24,196,N235LB,17.356815,-107.733948,18.273743,-107.547913,HBAL194,2020-06-23 10:16:23+00:00,103.842341,0 days 02:08:10.000000000,-1,0.000673,4.28


In [4]:
# We need the clusters to be close to stay points therefore they must have a min duration 
# which we can map to min_pts by computing the median frequency

bl['time_since_last_record'] = pd.to_timedelta(bl['time_since_last_record'])
freq_median = bl['time_since_last_record'].median()

print('Median is: ' + str(freq_median))

# Let's generate some possible stay times
# We set the lower bound to 3 hours and as upper bound we set 48 hours
upper_bound = 48
lower_bound = 3

num_values = 20

step_size = (upper_bound - lower_bound) / (num_values-1)
stay_point_sizes = np.arange(lower_bound, upper_bound + step_size, step_size)

stay_point_sizes = np.around(stay_point_sizes, decimals=0, out=None)

stay_point_sizes = pd.to_timedelta(stay_point_sizes, unit='h')

# Convert durations to minPts using the median frequency
stay_point_sizes = stay_point_sizes / freq_median

stay_point_sizes

Median is: 0 days 00:01:00


Float64Index([ 180.0,  300.0,  480.0,  600.0,  720.0,  900.0, 1020.0, 1200.0,
              1320.0, 1440.0, 1620.0, 1740.0, 1860.0, 2040.0, 2160.0, 2340.0,
              2460.0, 2580.0, 2760.0, 2880.0],
             dtype='float64')

In [5]:
kms_per_radian = 6371.0088

eps = bl.loc[bl.max_distance == chosen_distance]['eps'].unique()[0]

for minpts in stay_point_sizes:

    dbscan_bl = StandardScaler().fit_transform(bl[['lat', 'lon']])
    
    db = DBSCAN(eps=eps, min_samples=minpts, algorithm='ball_tree', metric='haversine') \
            .fit(np.radians(dbscan_bl))
        
    print(str(loon) + ': ' + str(np.unique(db.labels_)) + '; eps=' + str(eps) \
            + '; min_pts= ' + str(minpts))
    
    bl['cluster'] = db.labels_
    bl['minpts'] = minpts

    # Save to file   
    bl.to_csv( '../../../../og_data/DBSCAN/minptsRange/' + str(loon) \
                                  + '/CL-' + loon \
                                  + '-' + 'mp_' + str(minpts) + '-eps_' + str(eps) \
                                  + '.csv', index=False, encoding='utf-8-sig')

N235LB: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28 29 30 31 32 33 34]; eps=0.0006732801380231223; min_pts= 180.0
N235LB: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23 24 25 26 27 28]; eps=0.0006732801380231223; min_pts= 300.0
N235LB: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22
 23]; eps=0.0006732801380231223; min_pts= 480.0
N235LB: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12]; eps=0.0006732801380231223; min_pts= 600.0
N235LB: [-1  0  1  2  3  4  5  6  7  8  9 10 11 12]; eps=0.0006732801380231223; min_pts= 720.0
N235LB: [-1  0  1  2  3  4  5  6  7  8  9]; eps=0.0006732801380231223; min_pts= 900.0
N235LB: [-1  0  1  2  3  4]; eps=0.0006732801380231223; min_pts= 1020.0
N235LB: [-1  0  1  2]; eps=0.0006732801380231223; min_pts= 1200.0
N235LB: [-1  0  1  2]; eps=0.0006732801380231223; min_pts= 1320.0
N235LB: [-1  0  1  2]; eps=0.0006732801380231223; min_pts= 1440.0
N235LB: [-1  0]; ep